In [6]:
import numpy as np
import torch
import sys
from pathlib import Path
import importlib

import pandas as pd
import torchaudio

# module_path = str(Path.cwd().parents[0] / "utils")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if module_path not in sys.path:
#     sys.path.append(module_path)
module_path = str(Path.cwd().parents[0] / "network_models/w2v_emotion_model")
if module_path not in sys.path:
    sys.path.append(module_path)
# module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
# if module_path not in sys.path:
#     sys.path.append(module_path)

import utils.wav2Vec_utils as w2vU
import utils.audio_dataset_utils as ADU
import network_models.soundstream_lstm.LSTM_dataset as lds
import network_models.soundstream_lstm.CombinedEmoDataset_7_Emo as ced
import  network_models.w2v_emotion_model.custom_collator as cc
import network_models.w2v_emotion_model.custom_model as cm
import network_models.w2v_emotion_model.model_trainer as ct
import gc

model_name_or_path = "facebook/wav2vec2-large-960h-lv60-self"
pooling_mode = "mean"
device = "cuda"
emo_dataset = ced.CombinedEmoDataSet_7_emos(directory_tess="/home/ckwdani/Music/emotionDatasets/converted_mono/tess",
                                             directory_cafe="/home/ckwdani/Music/emotionDatasets/converted_mono/cafe",
                                             directory_ravdess="/home/ckwdani/Music/emotionDatasets/converted_mono/RAVDESS Audio_Speech_Actors_01-24",
                                             directory_mesd="/home/ckwdani/Music/emotionDatasets/converted_mono/mesd",
                                             device="cpu")

emo_dataset = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess", device=device)
processor, sr = w2vU.init_w2v2(num_labels=len(emo_dataset.label_list), label_list=emo_dataset.label_list, device=device)
newSet = lds.AudioEmotionTessWav2VecDataset(emo_dataset, processor= processor, sampling_rate=sr)
gc.collect()
model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device)
model.freeze_feature_extractor()


trainSet, evalSet = ADU.train_val_dataset(newSet, 0.1)
trainSet, evalSet = ADU.train_val_dataset(evalSet, 0.1)


data_collator = cc.DataCollatorCTCWithPadding(processor=processor, padding=True, num_labels=len(newSet.dataSet.label_list))

# trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=2, save_model_every=500, num_epochs=5001, model_path="../content/classifier/W2VClassifier/Nr1/", data_collator=data_collator)
# gc.collect()
# trainer.train()


In [ ]:
#model.load_state_dict(torch.load("../content/classifier/W2VClassifier/Nr1/emo_reco_950.pth"))
#model.from_pretrained()

In [8]:
importlib.reload(ct)
trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=4, save_model_every=100, num_epochs=5001, model_path="../content/classifier/W2VClassifier/Nr2_fullDs/", data_collator=data_collator, labelList=emo_dataset.label_list)
gc.collect()
trainer.train()

Epoch 1
-------------------------------
loss: 1.826298  [    0/  248]


/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/network_models/w2v_emotion_model/custom_model.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = softmax(y)


7
          accuracy  precision  recall   support
  angry     0.786     0.000     0.000    6
disgust     0.857     0.000     0.000    4
   fear     0.821     0.000     0.000    4
  happy     0.821     0.000     0.000    5
neutral     0.464     0.167     1.000    3
     ps     0.786     0.200     0.333    3
    sad     0.750     0.000     0.000    3
                                         28

 
    avg     0.755     0.052     0.190    

Test Error: 
 Accuracy: 14.3%, Avg loss: 1.964465 

Epoch 2
-------------------------------
loss: 1.994363  [    0/  248]
7
          accuracy  precision  recall   support
  angry     0.786     0.000     0.000    6
disgust     0.857     0.000     0.000    4
   fear     0.893     1.000     0.250    4
  happy     0.821     0.000     0.000    5
neutral     0.500     0.077     0.333    3
     ps     0.786     0.333     1.000    3
    sad     0.714     0.000     0.000    3
                                         28

 
    avg     0.765     0.201     0.226  

KeyboardInterrupt: 